In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

In [39]:
path="Keyword6"

In [3]:
PositionCTR=pd.read_csv("CTR10.csv")
Pdata= pd.read_csv('AuctionData.csv')

In [40]:
GFPK1=pd.read_csv(path+"\BidMatrixGFP.csv")
QualityScore=pd.read_csv(path+"\QualityScore.csv")
ClickRate=pd.read_csv(path+"\Clickrate.csv")
BidData=pd.read_csv(path+"\BidArray.csv")
GFP=np.array(GFPK1)

In [5]:
class Slot:
    CTR=-1
    Owner="NoName"
    impression=0
    click=0
    Cost=0

    def __init__(self, Ctr,imp,c):
        self.CTR=Ctr
        self.impression=imp
        self.click=c

In [6]:
class Bid:
    bid=-1
    bidder="NoName"
    quality=1
#     trueValue=0
    
    def __init__(self, Bidder,Bid,qua):
        self.bid=Bid
        self.bidder=Bidder
        self.quality=qua
#         self.trueValue=truevalue

In [7]:
class Auction:
    KeywordID=-1
    bids=[]
    
    def __init__(self,K_ID,bid):
        self.bids=[]
        self.KeywordID = K_ID
        for i in range(len(bid)):
            self.bids.append(bid[i])
# VCG Mechanism
    def VCG(self,slots):
        length = len(slots)
        
# Sort out bids through Score=Bid times Quality 
        for i in range(len(slots)):
            sum=0
            
            for j in range(i+1,len(slots)+1):
                if j!=(len(slots)):
                    sum+=self.bids[j].bid*(slots[j-1].CTR - slots[j].CTR)
                else:
                    sum+=self.bids[j].bid*(slots[j-1].CTR - 0)
                    
            slots[i].Cost=sum*(1/slots[i].CTR)
        
        for i in range(0, length):
            slots[i].Owner = self.bids[i].bidder
            
# GSP Mechansim  
    def GSP(self,slots):
        length = len(slots)
        
        for i in range(len(self.bids)):
            for j in range(i+1,len(self.bids)):
                if (self.bids[i].bid*self.bids[i].quality)<(self.bids[j].bid*self.bids[j].quality):
                    temp=self.bids[i]
                    self.bids[i]=self.bids[j]
                    self.bids[j]=temp

        for i in range(length):
            slots[i].Cost =(self.bids[i+1].bid*self.bids[i+1].quality)/self.bids[i].quality
#             slots[i].Cost =self.bids[i+1].bid
            
        for i in range(0, length):
            slots[i].Owner = self.bids[i].bidder
            
            
    def GSPnq(self,slots):
        length = len(slots)
        
        for i in range(len(self.bids)):
            for j in range(i+1,len(self.bids)):
                if (self.bids[i].bid*self.bids[i].quality)<(self.bids[j].bid*self.bids[j].quality):
                    temp=self.bids[i]
                    self.bids[i]=self.bids[j]
                    self.bids[j]=temp

        for i in range(length):
            slots[i].Cost =(self.bids[i+1].bid*self.bids[i+1].quality)/self.bids[i].quality
#             slots[i].Cost =self.bids[i+1].bid
            
        for i in range(0, length):
            slots[i].Owner = self.bids[i].bidder
            
# GFP Mechansim           
    def GFP(self,slots):
        length=len(slots)
        
        for i in range(length):
            slots[i].Cost=self.bids[i].bid
        
        for i in range(length):
            slots[i].Owner = self.bids[i].bidder


# OPT Mechansim
#     def OPT(self,slots):
#         length=len(slots)
        
#         for i in range(length):
#             slots[i].Cost=self.bids[i].bid
        
#         for i in range(length):
#             slots[i].Owner = self.bids[i].bidder

In [8]:
def SortBid(Bid):
    for i in range(len(Bid)):
        for j in range(i+1,len(Bid)):
            if Bid[i][1]<Bid[j][1]:
                Bt=Bid[j][1]
                At=Bid[j][0]
                Bid[j][1]=Bid[i][1]
                Bid[j][0]=Bid[i][0]
                Bid[i][1]=Bt
                Bid[i][0]=At
    return Bid

In [9]:
def FindQScore(BidderName,Quality):
    for i in range(len(Quality)):
        if Quality['BidderID'][i]==BidderName:
            return Quality['Quality'][i]

In [10]:
def TrueValue(Bidder,List):
    for i in range(len(List)):
        if List['BidderID'][i]==Bidder:
            return List['Price'][i]

In [11]:
def CalculateClick(position,bidder,ClickRate,pCTR):
    for i in range(len(ClickRate)):
        if bidder==ClickRate['Bidder'][i]:
            CR=ClickRate['ClickRate'][i]
            break
    return pCTR['Impression'][position]*CR

In [12]:
KeyWord="123"

In [41]:
SumB=0
SumE=0
SumC=0
for j in range(20):
    BidO=np.empty((20,2),dtype='object')
    for i in range(20):
        BidO[i][0]=GFP[i][0]
        BidO[i][1]=GFP[i][j+1]
    BidO=SortBid(BidO)

    bidList=[]
    for i in range(20):
        q=FindQScore(BidO[i][0],QualityScore)
        b=Bid(BidO[i][0],BidO[i][1],q)
        bidList.append(b)

    slotList=[]
    for i in range(10):
        s=Slot(PositionCTR['ClickThroughRate'][i],PositionCTR['Impression'][i],PositionCTR['Click'][i])
        slotList.append(s)
    
    auctionGFP = Auction(KeyWord,bidList)
    auctionGFP.GFP(slotList)
    print("Round %d"%(j+1))
    print(" %12s   %8s               %8s                    %8s   Click\n" % ("CTR", "Cost (per click)", "Bidder ID","Expected Utility(Revenue)"))
    for i,slot in enumerate(slotList):
        ClickTime=CalculateClick(i,slot.Owner,ClickRate,PositionCTR)
        SumE+=ClickTime*float(slot.Cost)
        SumB+=ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost)
        SumC+=ClickTime
        print("slot %2.0f:  %5.4f    %8.2f      %s         %10.0f            %8.0f" % (i+1, float(slot.CTR), float(slot.Cost), slot.Owner,ClickTime*(TrueValue(slot.Owner,BidData)-slot.Cost),ClickTime))
SumB/=20
SumE/=20
SumC/=20

Round 1
          CTR   Cost (per click)               Bidder ID                    Expected Utility(Revenue)   Click

slot  1:  0.0186    22440.00      ae30807c-5316-42ee-a031-589fdb08e4ba         1153239820              205569
slot  2:  0.0102     8000.00      52852966-92dd-43ac-8435-01b9e6021c7c                  0                   0
slot  3:  0.0073     3912.00      6524d297-40d9-4a96-90c0-0d5aa0c61da6          372618891              381001
slot  4:  0.0049     3000.00      54bff083-a345-436d-99f4-3e70f8969567           43747974               58331
slot  5:  0.0043     1600.00      a2860a55-f8a1-4d62-8edc-49fb88661adc           47506277              118766
slot  6:  0.0040     1600.00      ebd4f3cf-2d58-41d9-a848-e3db688ba491           96990368              242476
slot  7:  0.0032     1200.00      b5f02143-b63e-4062-bb82-0624909e55ac           41960374              139868
slot  8:  0.0028      800.00      9eb5a22f-1870-42b1-aaae-2faa5235ae34           57719595              288598
s

In [42]:
SumB

6591278815.735682

In [43]:
SumE

2260418598.524042

In [44]:
SumC

1525709.5523332427

In [45]:
SW=SumB+SumE
SW

8851697414.259724